In [1]:
import re
# download data from from Google Drive
!gdown 1uml6lt5diPDjBnJYVECBrafVJEISRU2-

Downloading...
From: https://drive.google.com/uc?id=1uml6lt5diPDjBnJYVECBrafVJEISRU2-
To: /content/Manchester-morph+changes_utterancesremoved.cha
100% 25.3M/25.3M [00:00<00:00, 35.7MB/s]


In [2]:
# read the file in to a list of tokens
tokens=[]
filename="Manchester-morph+changes_utterancesremoved.cha"
with open(filename) as file:
    for line in file:
      if re.match("^\*MOT",line):
          line=re.sub('\*[A-Z]+\:','', line)
          line = "{ " + line + " } "
          tokens.extend(line.split())
tokens=tokens[0:250000]

In [7]:
# Select context tokens based on frequency
from collections import Counter
counts=Counter(tokens)
# Sort the tuples. The reverse argument instructs to put most frequent first rather than last (which is the default)
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
so=list(zip(*so))[0]
# Select the firs 3500 words in the list
#vocab_size = 3500
type_list = so
#type_list=so[0:vocab_size]
vocab_size = len(type_list)
token_count = len(tokens)

In [8]:
from collections import defaultdict
import numpy as np
type_count = len(type_list)
targets = []
positions = []
contextwordcounts = []
d = defaultdict(lambda: [0] * vocab_size)
window_size = 2
M = np.zeros((type_count, 4, type_count))

for i, word in enumerate(tokens):
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = tokens[begin: end + 1]
            # Remove the target word from its own window
            context.remove(tokens[i])
            # Iterate over the window for this target word
            current_row = type_list.index(word)

            position = 0
            for token in context:

                if type_list.count(token) > 0:
                  # Find the ID and hence the list index for the current token
                  token_id = type_list.index(token)
                  # Add 1 to the current context word dimension for the current target word
                  d[(word,position)][token_id] += 1
                  M[current_row][position][token_id] += 1
                position += 1


In [9]:
# Convert to a hierachical data frame
import pandas as pd
df=pd.DataFrame(d)
df.index = type_list

In [10]:
df

{        here's        baby                   .  \
                            0      1      0   1  2    0   1    2    3     0   
{                           0      0      7  98  0  115  44    0    0  8275   
}                           0  36086     98   0  0   44   0    0  179     0   
.                       22370      0      0   0  1    0   0  111   71     0   
?                       12968      0      0   0  0    0   0   64   82     0   
you                         0      0      0   0  0   11   1    1    2   328   
...                       ...    ...    ...  .. ..  ...  ..  ...  ...   ...   
hears                       0      0      0   0  0    0   0    0    0     0   
grownup                     0      0      0   0  0    0   0    0    0     0   
Mr_Red_Man's                0      0      0   0  0    0   0    0    0     0   
Ring_a_ring_o(f)_roses      0      0      0   0  0    0   0    0    0     0   
chance                      0      0      0   0  0    0   0    0    0     0   

                        ... Ring_a_ring_o(f)_roses    chance          travels  \
                        ...                      2  3      0  1  2  3       0   
{                       ...                      0  0      0  0  0  0       0   
}                       ...                      0  1      0  0  0  0       0   
.                       ...                      1  0      0  0  0  0       0   
?                       ...                      0  0      0  0  0  0       0   
you                     ...                      0  0      0  0  0  0       0   
...                     ...                    ... ..    ... .. .. ..     ...   
hears                   ...                      0  0      0  0  0  0       0   
grownup                 ...                      0  0      0  0  0  0       0   
Mr_Red_Man's            ...                      0  0      0  0  0  0       0   
Ring_a_ring_o(f)_roses  ...                      0  0      0  0  0  0       0   
chance                  ...                      0  0      0  0  0  0       0   

                                 
                        1  2  3  
{                       0  0  0  
}                       0  0  2  
.                       0  2  0  
?                       0  0  0  
you                     0  0  0  
...                    .. .. ..  
hears                   0  0  0  
grownup                 0  0  0  
Mr_Red_Man's            0  0  0  
Ring_a_ring_o(f)_roses  0  0  0  
chance                  0  0  0  

[3669 rows x 14676 columns]

In [12]:
M.shape

(3669, 4, 3669)